In [6]:
def lire_fichier(chemin):
    with open("/home/rundimeco/Documents/corpora/corpus_paralleles/"+chemin, encoding="utf-8") as f:
        chaine = f.read()
    return chaine
import glob


In [2]:
import spacy
with open("index.json") as f:
    index = json.load(f)
nlp = spacy.load("fr_core_news_sm")

/home/rundimeco/anaconda3/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


FileNotFoundError: [Errno 2] No such file or directory: 'index.json'

In [7]:
#input: liste textes et requête
liste_textes = [
  "corpus_multi/fr/appr/2009-05-07_celex_IP-09-710.fr.html",
  "corpus_multi/fr/appr/2009-07-13_celex_IP-09-1118.fr.html",
  "corpus_multi/fr/appr/2009-09-28_celex_IP-09-1364.fr.html",
  "corpus_multi/fr/appr/2009-11-12_celex_IP-09-1703.fr.html",
  "corpus_multi/fr/appr/2009-11-10_celex_IP-09-1688.fr.html"
]


In [10]:
corpus = []
for chemin in liste_textes:
    chaine = lire_fichier(chemin)
    corpus.append(chaine)

requete = "faire baisser prix"
corpus.append(requete)
#vectoriser
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
#print(X)
#similarité
from sklearn.metrics.pairwise import cosine_similarity
import json
#for doc in corpus:
#    print(doc)
matrix = cosine_similarity(X.toarray())
print(matrix)
#output: textes par ordre décroissant de pertinence

[[1.         0.54185457 0.59520573 0.55809128 0.61178602 0.03035097]
 [0.54185457 1.         0.57086039 0.52104137 0.63247129 0.00272773]
 [0.59520573 0.57086039 1.         0.5541309  0.64418027 0.01090097]
 [0.55809128 0.52104137 0.5541309  1.         0.59904276 0.0044195 ]
 [0.61178602 0.63247129 0.64418027 0.59904276 1.         0.02374134]
 [0.03035097 0.00272773 0.01090097 0.0044195  0.02374134 1.        ]]


In [29]:
similarite_requete = matrix[-1]
print(similarite_requete)
liste_res = []
cpt= 0
for similarite in similarite_requete[:-1]:#on regarde pas le dernier
    chemin = liste_textes[cpt]
    liste_res.append([similarite, chemin])
    cpt+=1
resultat_trie = sorted(liste_res, reverse=True)
print(json.dumps(resultat_trie, indent=2))


[0.03035097 0.00272773 0.01090097 0.0044195  0.02374134 1.        ]
[
  [
    0.030350969162175702,
    "corpus_multi/corpus_multi/fr/appr/2009-05-07_celex_IP-09-710.fr.html"
  ],
  [
    0.023741340112292657,
    "corpus_multi/corpus_multi/fr/appr/2009-11-10_celex_IP-09-1688.fr.html"
  ],
  [
    0.010900971822863807,
    "corpus_multi/corpus_multi/fr/appr/2009-09-28_celex_IP-09-1364.fr.html"
  ],
  [
    0.004419496020852154,
    "corpus_multi/corpus_multi/fr/appr/2009-11-12_celex_IP-09-1703.fr.html"
  ],
  [
    0.002727726737311629,
    "corpus_multi/corpus_multi/fr/appr/2009-07-13_celex_IP-09-1118.fr.html"
  ]
]


In [37]:
import re
def afficher_contextes(chaine, terme, taille_contexte = 30):
   match = re.search(terme, chaine)
   contexts = []
   while match is not None:
     #Les bornes gauche et droite autour du mot :
     gauche = max(match.start()-taille_contexte-1, 0)
     droite =  match.end()+1+taille_contexte
     contexts.append(chaine[gauche:droite])
     chaine = chaine[match.end():]
     match = re.search(terme, chaine)
   for c in contexts:
     print(c)

def afficher_contextes_mots(chaine, terme, taille_contexte = 5):
   doc = nlp(chaine)
   liste_mots = []
   liste_positions = []
   cpt = 0
   for token in doc:
     liste_mots.append(str(token))
     if terme == str(token):
        liste_positions.append(cpt)
     cpt+=1    
   contexts = []
   for position in liste_positions:
     #Les bornes gauche et droite autour du mot :
     gauche = max(position-taille_contexte, 0)
     droite =  position+1+taille_contexte
     contexts.append(liste_mots[gauche:droite])
   for c in contexts:
     print(" ".join(c))        

In [ ]:
mots_requete = requete.split()
for sim, chemin in resultat_trie:
    chaine = lire_fichier(chemin)
    print(chemin)
    print("-"*20)
    for mot in mots_requete:
        afficher_contextes(chaine, mot)
        afficher_contextes_mots(chaine, mot)
        dd=input("next?")

corpus_multi/corpus_multi/fr/appr/2009-05-07_celex_IP-09-710.fr.html
--------------------

régulation. Cela permettra de faire converger les tarifs de termin
régulation . Cela permettra de faire converger les tarifs de terminaison
next?
téléphonie à travers l'UE fera baisser les prix des appels
vocaux dan
à travers l' UE fera baisser les prix des appels 

next?
élimination des distorsions de prix entre les opérateurs
de téléph
 travers l'UE fera baisser les prix des appels
vocaux dans et entr
erminaison qui ne sont pas des prix fixés par le marché,
mais qui 
xe-mobile ou offres
d'appels à prix compétitifs). 
Les tarifs de t
L' élimination des distorsions de prix entre les opérateurs 
 de
l' UE fera baisser les prix des appels 
 vocaux dans
qui ne sont pas des prix fixés par le marché ,
offres 
 d' appels à prix compétitifs ) . 
 Les
next?
corpus_multi/corpus_multi/fr/appr/2009-11-10_celex_IP-09-1688.fr.html
--------------------
 mais nous pouvons bien sûr en faire davantage, et telle e